## Experiments
### Running finetuning using entire VGG model VS using precomputed conv. model outputs
### Conclusion: Precomputing the conv. model output seems fine... Final val_losss values 
### are very similar.

### Loading and Running the finetuning in plain old batches through the entire model

In [1]:
from Executor import *
# Reset the model, and run same finetuning using precomputed conv. model outputs. See if they match...
executor = ExecutorBuilder().\
    with_runID("first").\
    and_().\
    with_Vgg16().\
    and_().\
    train_batch_size(128). \
    and_(). \
    learn_rate(0.001).\
    and_().\
    data_on_path("../data/full/").\
    build()

Using Theano backend.
Using gpu device 0: Tesla K80 (CNMeM is disabled, cuDNN 5103)
/home/ubuntu/anaconda2/lib/python2.7/site-packages/theano/sandbox/cuda/__init__.py:600: UserWarning: Your cuDNN version is more recent than the one Theano officially supports. If you see any problems, try updating Theano or downgrading cuDNN to version 5.
  warnings.warn(warn)


Pretrained Vgg16 model loaded.
Found 6255 images belonging to 3 classes.
Found 1549 images belonging to 3 classes.
initialized training data from: ../data/full/train
initialized validation data from: ../data/full/valid
found number of softmax classes: 3


In [2]:
executor.precompute_conv_model_outputs()


precomputing conv. model outputs..
Found 6255 images belonging to 3 classes.
Found 1549 images belonging to 3 classes.
('models saved to files: ', 'precomputed_trn_features.first.h5', ' and ', 'precomputed_val_features.first.h5')
done.


<Executor.Executor instance at 0x7f9fa72eff80>

### Performing same operation using precomputed conv. model outputs

In [3]:
executor.load_precomputed_conv_models()

loading precomputed conv. outputs...
done...


<Executor.Executor instance at 0x7fce4b696d88>

In [3]:
# Get the linear model 
linearModel = executor.get_rescaled_fc_model(new_dropout=executor.dropout)

# Set all layers except the last as being trainable
i=0
for layer in linearModel.layers:
    i+=1
    if(i!=7):
        layer.trainable=False
    if layer.trainable:
        print("found trainable layer: ",layer)


getting rescaled fc model...
found dense layer. Distributing scaled weights..
found dense layer. Distributing scaled weights..
found dense layer. Distributing scaled weights..
found dense layer. Distributing scaled weights..
found dense layer. Distributing scaled weights..
found dense layer. Distributing scaled weights..
found dense layer. Distributing scaled weights..
done...
('updating dropout from: ', 0.5, ' to: ', 0.5)
('found trainable layer: ', <keras.layers.core.Dense object at 0x7f9f9d687250>)


In [4]:
# try fitting the model. Used an extremely small learning rate... still no avail
linearModel.compile(optimizer=Adam(lr=0.001), loss='categorical_crossentropy', metrics = ['accuracy'])
linearModel.fit(executor.train_precomputed, executor.train_labels, batch_size=executor.batch_size,
                nb_epoch= 5, validation_data=(executor.val_precomputed, executor.val_labels))

Train on 6255 samples, validate on 1549 samples
Epoch 1/5
6255/6255 [==============================] - 3s - loss: 1.7946 - acc: 0.4334 - val_loss: 1.0027 - val_acc: 0.5281
Epoch 2/5
6255/6255 [==============================] - 3s - loss: 1.4086 - acc: 0.4884 - val_loss: 1.0035 - val_acc: 0.5526
Epoch 3/5
6255/6255 [==============================] - 3s - loss: 1.2796 - acc: 0.5044 - val_loss: 0.9618 - val_acc: 0.5642
Epoch 4/5
6255/6255 [==============================] - 3s - loss: 1.1832 - acc: 0.5175 - val_loss: 0.9027 - val_acc: 0.5578
Epoch 5/5
6255/6255 [==============================] - 3s - loss: 1.1090 - acc: 0.5276 - val_loss: 0.8867 - val_acc: 0.5752


In [5]:
linearModel.compile(optimizer=Adam(lr=0.0001), loss='categorical_crossentropy', metrics = ['accuracy'])
linearModel.fit(executor.train_precomputed, executor.train_labels, batch_size=executor.batch_size,
                nb_epoch= 5, validation_data=(executor.val_precomputed, executor.val_labels))

Train on 6255 samples, validate on 1549 samples
Epoch 1/5
6255/6255 [==============================] - 3s - loss: 1.0088 - acc: 0.5575 - val_loss: 0.8544 - val_acc: 0.5875
Epoch 2/5
6255/6255 [==============================] - 3s - loss: 1.0063 - acc: 0.5552 - val_loss: 0.8489 - val_acc: 0.5913
Epoch 3/5
6255/6255 [==============================] - 3s - loss: 0.9868 - acc: 0.5484 - val_loss: 0.8478 - val_acc: 0.5946
Epoch 4/5
6255/6255 [==============================] - 3s - loss: 0.9590 - acc: 0.5664 - val_loss: 0.8429 - val_acc: 0.6004
Epoch 5/5
6255/6255 [==============================] - 3s - loss: 0.9705 - acc: 0.5672 - val_loss: 0.8455 - val_acc: 0.5875


In [6]:
linearModel.compile(optimizer=Adam(lr=0.00001), loss='categorical_crossentropy', metrics = ['accuracy'])
linearModel.fit(executor.train_precomputed, executor.train_labels, batch_size=executor.batch_size,
                nb_epoch= 5, validation_data=(executor.val_precomputed, executor.val_labels))

Train on 6255 samples, validate on 1549 samples
Epoch 1/5
6255/6255 [==============================] - 3s - loss: 0.9596 - acc: 0.5717 - val_loss: 0.8434 - val_acc: 0.5913
Epoch 2/5
6255/6255 [==============================] - 3s - loss: 0.9468 - acc: 0.5695 - val_loss: 0.8430 - val_acc: 0.5913
Epoch 3/5
6255/6255 [==============================] - 3s - loss: 0.9572 - acc: 0.5661 - val_loss: 0.8421 - val_acc: 0.5939
Epoch 4/5
6255/6255 [==============================] - 3s - loss: 0.9614 - acc: 0.5616 - val_loss: 0.8418 - val_acc: 0.5952
Epoch 5/5
6255/6255 [==============================] - 3s - loss: 0.9622 - acc: 0.5709 - val_loss: 0.8414 - val_acc: 0.5959


In [7]:
linearModel.compile(optimizer=Adam(lr=0.000001), loss='categorical_crossentropy', metrics = ['accuracy'])
linearModel.fit(executor.train_precomputed, executor.train_labels, batch_size=executor.batch_size,
                nb_epoch= 5, validation_data=(executor.val_precomputed, executor.val_labels))

Train on 6255 samples, validate on 1549 samples
Epoch 1/5
6255/6255 [==============================] - 3s - loss: 0.9379 - acc: 0.5762 - val_loss: 0.8414 - val_acc: 0.5959
Epoch 2/5
6255/6255 [==============================] - 3s - loss: 0.9477 - acc: 0.5655 - val_loss: 0.8414 - val_acc: 0.5972
Epoch 3/5
6255/6255 [==============================] - 3s - loss: 0.9586 - acc: 0.5651 - val_loss: 0.8414 - val_acc: 0.5965
Epoch 4/5
6255/6255 [==============================] - 3s - loss: 0.9541 - acc: 0.5682 - val_loss: 0.8414 - val_acc: 0.5978
Epoch 5/5
6255/6255 [==============================] - 3s - loss: 0.9677 - acc: 0.5600 - val_loss: 0.8413 - val_acc: 0.5965


In [8]:
executor.save_model_to_file()

<Executor.Executor instance at 0x7f9fa72eff80>